In [8]:
from flask import Flask, render_template, request, jsonify
import pandas as pd

app = Flask(__name__)

# Read the CSV file directly from a URL
url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv'
df = pd.read_csv(url)

@app.route('/', methods=['GET'])
def index():
    # Get column names for the dropdown list
    columns = df.columns.tolist()
    # List of available aggregation operations
    operations = ['sum', 'mean', 'max', 'min', 'count']
    return render_template('index.html', columns=columns, operations=operations)

@app.route('/apply', methods=['POST'])
def apply():
    # Get the selected group keys
    groupby_columns = request.form.getlist('groupby[]')

    # Validate that at least one group key is selected
    if not groupby_columns:
        return jsonify(error='Please select at least one group key.')

    # Get the selected columns for aggregation
    selected_columns = request.form.getlist('columns[]')

    # Validate that groupby columns and selected columns are different
    if set(groupby_columns).intersection(set(selected_columns)):
        return jsonify(error='Please select different columns for group keys and operations.')

    # Group by the selected columns
    df_grouped = df.groupby(groupby_columns)

    # Get the selected operations
    operation_names = request.form.getlist('operation[]')

    # Perform the selected aggregation operations
    df_grouped = df_grouped.agg({col: operation_names for col in selected_columns})

    # Convert DataFrame to an HTML table with commas and two decimal points
    table = df_grouped.to_html(float_format=lambda x: '{:,.2f}'.format(x), table_id="resultTable")


    return jsonify(table=table)

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jun/2023 22:53:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:53:18] "POST /apply HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:53:19] "POST /apply HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:53:19] "POST /apply HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:53:19] "POST /apply HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:53:19] "POST /apply HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:53:20] "POST /apply HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:53:21] "POST /apply HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:53:26] "POST /apply HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:58:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 22:59:01] "POST /apply HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2023 21:10:37] "POST /apply HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2023 21:10:39] "POST /apply HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2023 21:10:42] "POST /apply HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2023 2